In [16]:
import sys
import os

src_path = os.path.abspath('..')  # go from feature_extraction → src
print("Adding to sys.path:", src_path)
sys.path.append(src_path)

import os
print(os.getcwd())


Adding to sys.path: /Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/src
/Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/src/feature_extraction


In [17]:
# first version of function
import re # regular expression module for word tokenization
from feature_extraction.features import n_words

def filler_word_ratio(text, filler_words=None):
    """
    Calculates the proportion of filler words in a text.
    """
    if isinstance(text, float):  # handle NaN
        text = ""

    # todo: why these filler words?
    if filler_words is None:
        filler_words = {"um", "uh", "err", "hmm", "like", "well", "you know", "i mean"} # custom list of common filler words / phrases

    total_words = n_words(text)

    # tokenize text to lowercase words for phrase-matching
    # todo: isn't this the same as n_words?
    words = [w.lower() for w in re.findall(r"\b\w+\b", text)] # gets actual words without punctuation and symbols

    # count filler words
    filler_count = 0
    i = 0
    while i < len(words): # loop through text word by word, index-based
        matched = False
        for phrase in sorted(filler_words, key=lambda x: -len(x.split())):  # check multi-word fillers first
            phrase_tokens = phrase.split() # slice current section of words list
            if words[i:i + len(phrase_tokens)] == phrase_tokens: # if slice matches phrase, it's counted as a filler
                filler_count += 1
                i += len(phrase_tokens) # skips ahead len(phrase_tokens) when matched, so that it doesn't double-count
                matched = True
                break
        if not matched:
            i += 1

    return filler_count / total_words if total_words > 0 else None # ratio of filler words to total words (None if text is empty)


In [18]:
def filler_word_ratio2(text, filler_words=None):
    if isinstance(text, float):
        text = ""

    if filler_words is None:
        filler_words = {"um", "uh", "err", "hmm", "like", "well", "you know", "i mean"}

    total_words = n_words(text)
    if total_words == 0:
        return None

    text_cleaned = re.sub(r'[^\w\s]', '', text.lower())  # basic punctuation removal
    filler_count = sum(
        len(re.findall(r'\b' + re.escape(phrase) + r'\b', text_cleaned))
        for phrase in filler_words
    )
    # Note: overlapping filler expressions may be double-counted if filler_words contain subphrases

    return filler_count / total_words

In [27]:
text = "Well, I was going to the store, and you know, I saw this guy who looked familiar. Um, I couldn’t quite remember where I knew him from. Like, he had this face that just stuck in my mind. So I mean, maybe I saw him at a concert or something? Err, anyway, I kept walking, but uh, I couldn’t stop thinking about it. Hmm, that ever happen to you? It’s so weird when your brain just won’t let go of something. I mean, it’s probably nothing, but well, now I want to figure it out."
filler_word_ratio(text)

0.10526315789473684

In [28]:

filler_word_ratio2(text)

0.10526315789473684

In [29]:
print("Total words:", n_words(text))


Total words: 95
